<a href="https://colab.research.google.com/github/pinilDissanayaka/Psychology-RAG-Fusion/blob/main/Notebook_RAG_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
from langchain_groq import ChatGroq
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

In [12]:
def getLlm():
  os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
  llm=ChatGroq(model_name='llama-3.1-70b-versatile',
              temperature=0.6)
  return llm


In [14]:
llm=getLlm()
llm.invoke('hi')

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_time': 0.04, 'prompt_time': 0.005964215, 'queue_time': None, 'total_time': 0.045964215}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'stop', 'logprobs': None}, id='run-974b339b-cb68-4d95-b10e-c3f6a6177ed7-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21})

In [20]:
os.environ['PINECONE_API_KEY']=userdata.get('PINECORN_API_KEY')
pinecone=Pinecone()

TypeError: 'NoneType' object is not callable